In [1]:
! pip install transformers datasets
! pip install sentencepiece
! pip install evaluate
! pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.1 MB/s eta 0:00:00
Looking in index

#Подготовка датасета

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import json
import re
import pandas as pd 
wc = []
reviews = {'review' : [], 'label' : []}
for line in open("/content/drive/MyDrive/datasets/IMDB_reviews.json"):
    review = json.loads(line)
    reviews['review'].append(review['review_text'])
    reviews['label'].append(int(review['is_spoiler']))
    wc.append(len(re.findall(r'\w+', review['review_text'])))

In [4]:
r_lists = list(zip(reviews['review'], reviews['label'], wc))

df = pd.DataFrame(r_lists, columns=['review', 'label', 'word_count'])

####Создадим сбалансированный датасет из рецензий с <200 слов

In [5]:
from datasets import load_dataset, load_metric, Dataset
def sampling_k_elements(group, k=5000):
    if len(group) < k:
        return group
    return group.sample(k)
reviews_shortened = df[df['word_count']<200].groupby('label').apply(sampling_k_elements).reset_index(drop=True)
reviews_shortened_dict = reviews_shortened.to_dict('list')
ds = Dataset.from_dict(reviews_shortened_dict)
ds_split= ds.train_test_split(test_size=0.2)


In [6]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)


def tokenize_function(examples):
    return tokenizer(examples["review"], max_length=512,padding="max_length",truncation=True)


tokenized_datasets = ds_split.map(tokenize_function,batched = True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

In [11]:
train_dataset

Dataset({
    features: ['review', 'label', 'word_count', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8000
})

###Подбор параметров

In [8]:
from transformers import TrainingArguments

test_training_args = TrainingArguments( 
    output_dir='/content/drive/MyDrive/test_results',
    save_total_limit=2,
    load_best_model_at_end=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps = 2,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_steps=50,
    save_steps=50,
    warmup_steps=50,
    optim="adafactor",
    
)

KeyboardInterrupt: ignored

<a id='trainer'></a>

In [14]:
def model_init():
    return  XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)

In [15]:
new_train_dataset = tokenized_datasets["train"].shard(index=1, num_shards=5)

In [16]:
from transformers import Trainer, XLNetForSequenceClassification
n_trainer = Trainer(
    model_init=model_init,
    args=test_training_args,
    train_dataset=new_train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [17]:
best_run = n_trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2023-03-23 13:39:44,202] A new study created in memory with name: no-name-455c9ee6-b9c5-4b4a-8ec1-fd9ecf2c5240
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.

Step,Training Loss,Validation Loss,Accuracy
50,No log,0.723386,0.490000
100,No log,0.611732,0.670000
150,No log,0.592897,0.681000
200,No log,0.638623,0.679500


[I 2023-03-23 13:47:05,397] Trial 0 finished with value: 0.6795 and parameters: {'learning_rate': 3.975115756654692e-05, 'num_train_epochs': 4, 'seed': 18, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.6795.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are

Step,Training Loss,Validation Loss


[I 2023-03-23 13:48:56,232] Trial 1 finished with value: 0.547 and parameters: {'learning_rate': 8.624705445406433e-06, 'num_train_epochs': 1, 'seed': 3, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.6795.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are n

...

[I 2023-03-23 14:07:19,660] Trial 2 finished with value: 0.662 and parameters: {'learning_rate': 8.056091377147518e-05, 'num_train_epochs': 5, 'seed': 17, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.6795.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are n

Step,Training Loss,Validation Loss,Accuracy
50,No log,0.681990,0.547000
100,No log,0.603758,0.678000
150,No log,0.597526,0.689500


[I 2023-03-23 14:12:46,068] Trial 3 finished with value: 0.6895 and parameters: {'learning_rate': 1.0639214874359341e-05, 'num_train_epochs': 3, 'seed': 36, 'per_device_train_batch_size': 16}. Best is trial 3 with value: 0.6895.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and ar

Step,Training Loss,Validation Loss,Accuracy
50,No log,0.680429,0.578500
100,No log,0.674301,0.571500
150,No log,0.668315,0.596000
200,No log,0.664966,0.600000
250,No log,0.661429,0.612000
300,No log,0.659634,0.614500


[I 2023-03-23 14:20:03,576] Trial 4 finished with value: 0.6145 and parameters: {'learning_rate': 1.922445468764568e-06, 'num_train_epochs': 3, 'seed': 7, 'per_device_train_batch_size': 8}. Best is trial 3 with value: 0.6895.


In [18]:
best_run

BestRun(run_id='3', objective=0.6895, hyperparameters={'learning_rate': 1.0639214874359341e-05, 'num_train_epochs': 3, 'seed': 36, 'per_device_train_batch_size': 16}, run_summary=None)

### Обучение

In [9]:
from transformers import XLNetForSequenceClassification

model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments( 
    output_dir='/content/drive/MyDrive/results',
    save_total_limit=2,
    load_best_model_at_end=True,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    evaluation_strategy='steps',
    learning_rate= 1.0639214874359341e-05,
    eval_steps=250,
    save_steps=250,
    warmup_steps=100,
    optim="adafactor",
    
)

In [11]:

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
250,No log,0.595833,0.665500
500,0.639900,0.601043,0.668500
750,0.639900,0.577832,0.701500
1000,0.565200,0.587039,0.705500
1250,0.565200,0.627091,0.707500
1500,0.500000,0.609132,0.708500
1750,0.500000,0.638860,0.707500
2000,0.439500,0.660051,0.709500


TrainOutput(global_step=2000, training_loss=0.5361532897949218, metrics={'train_runtime': 8658.8506, 'train_samples_per_second': 3.696, 'train_steps_per_second': 0.231, 'total_flos': 9116168945664000.0, 'train_loss': 0.5361532897949218, 'epoch': 4.0})

In [15]:
trainer.evaluate()

{'eval_loss': 0.5778315663337708,
 'eval_accuracy': 0.7015,
 'eval_runtime': 247.0371,
 'eval_samples_per_second': 8.096,
 'eval_steps_per_second': 0.506,
 'epoch': 4.0}

In [16]:
trainer.save_model("/content/drive/MyDrive/model_final")

<a id='pytorch_native'></a>